In [ ]:
# First we need to import all of the packages we will be using!
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

# Color-blind friendly color cycle
CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=CB_color_cycle)

## Model

In [ ]:
#Our model

def model(states, t, L_on, L_off, R2_on, R2_off, L_amp, R2_amp):
    # Our "states" list has the state variables in the following order:
    # R1 (repressor 1), pr_a (active promoter), pr_R1 (promoter bound to R1), pr_R2 (promoter bound to R2), R1L (R1 bound to ligand), G (gene product)
    R1 = states[1]
    pr_a = states[0]
    pr_R1 = states[2]
    pr_R2 = states[3]
    R1L = states[4]
    G = states[5]
    
    # L (ligand) dynamics
    if (t > L_on and t < L_off):
        L = L_amp
    else:
        L = 0

    # R2 (repressor 2) dynamics
    if (t > R2_on and t < R2_off):
        R2 = R2_amp
    else:
        R2 = 0
    
    ### implement the change equations
    L_prime = -k_deg3 - k_b*R1L -k_a*R1*L - k_deg3*R1L
    R1_prime = -k_a*R1*L + k_b*R1L + k_deg3*R1L - k_c*R1*pr_a
    R2_prime = -k_deg2 - k_e*R2*pr_a + k_f*pr_R2
    pr_a_prime = -k_c*pr_a*R1 + k_d*pr_R1 - k_e*R2*pr_a + k_f*pr_R2
    pr_R1_prime = k_c*pr_a*R1 - k_d*pr_R1
    pr_R2_prime = k_e*R2*pr_a - k_f*pr_R2
    G_prime = k_syn*pr_a - k_deg1*G
    R1L_prime = k_a*R1*L - k_b*R1L - k_deg3*R1L
    ###

    statesprime = [R1_prime, pr_a_prime, pr_R1_prime, pr_R2_prime, R1L_prime, G_prime]
    return statesprime

In [ ]:
#Simulation of Model

#initial conditions
R1_0 = 
pr_a_0 = 
pr_R1_0 = 
pr_R2_0 = 
R1L_0 = 
G_0 = 

#parameters
Kd1 = 0.1
Kd2 = 0.1
k_a = 0.2     #(min)^-1
k_d = 0.05    #(min)^-1
k_syn = 0.05   #uM(min^1)
k_deg = 0.20  #(min)^-1
tau = 25      # min

#NFKB activation
L_on_time = 0
L_off_time = 60
R2_on_time = 0
R2_off_time = 60
L_amplitude = 1
R2_amplitude = 1

t_final=np.array([])
soln_final = np.empty((0, 6))
states0 = [R1_0, pr_a_0, pr_R1_0, pr_R2_0, R1L_0, G_0]
soln_hist = np.tile(states0, (1001, 1))

iterations = 25

for i in range(iterations):
    # solve ODE in one period of tau, using saved values of delayed variables
    t_index = np.linspace(0,tau,1001)
    solution = odeint(model,states0,t_index, args=(L_on_time, L_off_time, R2_on_time, R2_off_time, L_amplitude, R2_amplitude))
    
    # append new solution to array of old solution values
    t_final = np.append(t_final, (t_index[0:1000]+i*tau))
    soln_final = np.vstack((soln_final, solution[0:1000, :]))
    
    #update IFNAR on/off times, initial conditions, and solution history for next calculation
    NFKB_on1_time = NFKB_on1_time - tau
    NFKB_off1_time = NFKB_off1_time - tau
    NFKB_on2_time = NFKB_on2_time - tau
    NFKB_off2_time = NFKB_off2_time - tau
    states0 = solution[1000, :]
    soln_hist = solution